In [ ]:
! pip install mecab-python3 unidic-lite
import MeCab
import numpy as np
import pandas as p

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from matplotlib.projections.geo import Circle
df = pd.read_csv('/content/drive/Shareddrives/FY22_市場開発部IS_学生作業用/train_data.csv')
print(df)

niju = df[df['Label']=='◎']
maru = df[df['Label']=='○']
sikaku = df[df['Label']=='□']
sankaku = df[df['Label']=='▲']
batu = df[df['Label']=='×']
print(len(niju))
print(len(maru))
print(len(sikaku))
print(len(sankaku))
print(len(batu))

In [ ]:
sikaku_han_random = sikaku.sample(frac=0.5) # ランダムに半分を取り出し
sikakules2 = pd.concat([sikaku_han_random, df[df['Label'] != '□']]) # □が入っていないdfに上を連結

In [ ]:
df_mae = df[df.index<249738] # 前半
df_ato = df[df.index>=249738] # 後半

In [ ]:
# すべてのコメント取り出してtxtに吐き出す
tagger = MeCab.Tagger('-Owakati')

f = open('/content/drive/MyDrive/all_comment.txt', 'w')
for comment in df["Comment"]:
  comment = str(comment).replace("・", "").strip()
  comment = tagger.parse(comment) # ここで既に分かち済み
  print(comment, end="", file=f)
  # f.write(str(comment))
f.close()

In [ ]:
! pip install gensim
from gensim.models import word2vec

In [ ]:
# word2vecモデルの作成(再実行不要)
docs = word2vec.LineSentence("/content/drive/MyDrive/all_comment.txt")
model = word2vec.Word2Vec(docs, size=100, min_count=5, window=5, iter=3)
model.save("/content/drive/MyDrive/data_based.model")


In [ ]:
# 特定ワードに近い単語10個を出力
result = model.most_similar(positive="景気", topn=10)
print(result)

In [ ]:
tagger=MeCab.Tagger() # 上で記載済みだが念のため
data_based_model = word2vec.Word2Vec.load("/content/drive/MyDrive/data_based.model") # データから作成したモデルを読み込み
# general_model = word2vec.Word2Vec.load("/content/drive/MyDrive/general.model") # 汎用モデルを読み込み

hukushi_dic = {'特に':0.5, 'やや':-0.25, '少し':-0.25, 'あまり':-0.25, 'まだ':-0.25, 'かなり':0.5, '依然':0, '全く':-0.75, 'なかなか':-0.25, 'しばらく':-0.25, '更に':0.25, 'ほぼ':0, 'まだまだ':-0.25, 'ますます':0.25, 'どう':0, 'そう':1, '大変':0.5, 'よく':0.5, 'より':0.5, '余り':-0.25} # 副詞リスト
# ↓改変版
# hukushi_dic = {'特に':0.5, 'やや':-0.25, '少し':-0.25, 'あまり':-0.5, 'まだ':-0.25, 'かなり':0.75, '依然':0, '全く':-0.75, 'なかなか':-0.5, 'しばらく':-0.25, '更に':0.25, 'ほぼ':0, 'まだまだ':-0.25, 'ますます':0.5, 'どう':0, 'そう':0, '大変':0.5, 'よく':0.5, 'より':0.5, '余り':-0.25} # 副詞リスト
hukushilist = list(hukushi_dic.keys())
jodoshi_dic = {'ない':-1, 'ず':-1, 'なく':-1, 'なかっ':-1, 'なけれ':-1, 'ざる':-1} # 助動詞リスト
# jodoshi_dic = {} # 助動詞リスト

jodoshi_list = list(jodoshi_dic.keys())
n = 100 # ベクトルの次元

# ランダムなしのコメント→ベクトル関数
def comment_to_vec(text):
  adv = 0 # 副詞カウンター
  h_num = 0 # 特定副詞のレバレッジ
  j_num = 0 # 特定助動詞のレバレッジ
  vec = np.zeros(n, dtype=np.float64) # dtypeは仮にfloat64に設定
  node = tagger.parseToNode(text).next # BOS/EOS部分を削除するために1つずらし
  
  while node.next: # 同様に一つずらし
    word = node.surface
    hinshi = node.feature.split(",")[0]
    try:
      node_vec = data_based_model[word]
    except KeyError:
      # node_vec = general_model[word] # 汎用モデルを使うことがあれば
      node_vec = np.zeros(n)
    
    # print(word, hinshi, node_vec)
    
    if hinshi == "名詞":
      vec += node_vec

    elif hinshi == "動詞":
      if adv == 1: 
        node_vec = node_vec * (1+h_num) # 副詞のレバレッジを乗算
        adv = 0
      if node.next.feature.split(",")[0] == "助動詞" and node.next.surface in jodoshi_list:
        j_num = jodoshi_dic[node.next.surface]
        node_vec = node_vec * j_num # 助動詞のレバレッジを乗算
        
      vec += node_vec
        
    elif hinshi == "形容詞":
      if adv == 1:
        node_vec = node_vec * (1+h_num) # 副詞のレバレッジを乗算
        adv = 0
        
      vec += node_vec

    elif hinshi == "副詞":
      if word in hukushilist:
        h_num = hukushi_dic[word]
        adv = 1
      
      
    node = node.next
    
      
  return vec # 出力はdtype=float64のn次元ベクトル

    


print(comment_to_vec("私は体操をする。"))
print(comment_to_vec("私は特に体操をする。"))
print(comment_to_vec("私は体操をした。"))
print(comment_to_vec("私は体操をしなかった。"))

print(list(comment_to_vec("私は体操をした。")))
print(type(list(comment_to_vec("私は体操をした。"))))
print(len(list(comment_to_vec("私は体操をした。"))))

In [ ]:
tagger=MeCab.Tagger() # 上で記載済みだが念のため
data_based_model = word2vec.Word2Vec.load("/content/drive/MyDrive/data_based.model") # データから作成したモデルを読み込み
# general_model = word2vec.Word2Vec.load("/content/drive/MyDrive/general.model") # 汎用モデルを読み込み

hukushi_dic = {'特に':0.5, 'やや':-0.25, '少し':-0.25, 'あまり':-0.25, 'まだ':-0.25, 'かなり':0.5, '依然':0, '全く':-0.75, 'なかなか':-0.25, 'しばらく':-0.25, '更に':0.25, 'ほぼ':0, 'まだまだ':-0.25, 'ますます':0.25, 'どう':0, 'そう':1, '大変':0.5, 'よく':0.5, 'より':0.5, '余り':-0.25} # 副詞リスト
# ↓改変版
# hukushi_dic = {'特に':0.5, 'やや':-0.25, '少し':-0.25, 'あまり':-0.5, 'まだ':-0.25, 'かなり':0.75, '依然':0, '全く':-0.75, 'なかなか':-0.5, 'しばらく':-0.25, '更に':0.25, 'ほぼ':0, 'まだまだ':-0.25, 'ますます':0.5, 'どう':0, 'そう':0, '大変':0.5, 'よく':0.5, 'より':0.5, '余り':-0.25} # 副詞リスト
hukushilist = list(hukushi_dic.keys())
jodoshi_dic = {'ない':-1, 'ず':-1, 'なく':-1, 'なかっ':-1, 'なけれ':-1, 'ざる':-1} # 助動詞リスト
# jodoshi_dic = {} # 助動詞リスト

jodoshi_list = list(jodoshi_dic.keys())
n = 100 # ベクトルの次元

# ランダムありのコメント→ベクトル関数
def comment_to_vec_r(text):
  adv = 0 # 副詞カウンター
  h_num = 0 # 特定副詞のレバレッジ
  j_num = 0 # 特定助動詞のレバレッジ
  vec = np.zeros(n, dtype=np.float64) # dtypeは仮にfloat64に設定
  node = tagger.parseToNode(text).next # BOS/EOS部分を削除するために1つずらし
  
  while node.next: # 同様に一つずらし
    word = node.surface
    hinshi = node.feature.split(",")[0]
    rand = (1.05-0.95) * (np.random.rand()) + 0.95
    try:
      node_vec = (data_based_model[word]) * rand
    except KeyError:
      # node_vec = general_model[word] # 汎用モデルを使うことがあれば
      node_vec = np.zeros(n)
    
    # print(word, hinshi, node_vec)
    
    if hinshi == "名詞":
      vec += node_vec

    elif hinshi == "動詞":
      if adv == 1: 
        node_vec = node_vec * (1+h_num) # 副詞のレバレッジを乗算
        adv = 0
      if node.next.feature.split(",")[0] == "助動詞" and node.next.surface in jodoshi_list:
        j_num = jodoshi_dic[node.next.surface]
        node_vec = node_vec * j_num # 助動詞のレバレッジを乗算
        
      vec += node_vec
        
    elif hinshi == "形容詞":
      if adv == 1:
        node_vec = node_vec * (1+h_num) # 副詞のレバレッジを乗算
        adv = 0
        
      vec += node_vec

    elif hinshi == "副詞":
      if word in hukushilist:
        h_num = hukushi_dic[word]
        adv = 1
      
      
    node = node.next
    
      
  return vec # 出力はdtype=float64のn次元ベクトル

In [ ]:
# データセット作成(train)→csv出力
df = pd.read_csv('/content/drive/shareddrives/fy22_市場開発部is_学生作業用/train_data.csv')
df = df[['label', 'comment']]
print(df)
result_list = []

for data in zip(df['label'], df['comment']):
  result = []
  label_num = 0
  if data[0] == '◎':
    label_num = 1.0
  elif data[0] == '○':
    label_num = 0.5
  elif data[0] == '□':
    label_num = 0
  elif data[0] == '▲':
    label_num = -0.5
  elif data[0] == '×':
    label_num = -1.0
  result.append(label_num)
  comment_vec = list(comment_to_vec(str(data[1]))) # n次元の固有ベクトルをリストで出力 水増しデータ作る場合はcomment_to_vec_rを使用
  result += comment_vec
  result_list.append(result)
  # print(result)

result_df = pd.dataframe(result_list) # result_listをdataframeに変換

result_df.to_csv("/content/drive/mydrive/train_data_set1.csv")

In [ ]:
# データセット作成(test)→csv出力
df = pd.read_csv('/content/drive/Shareddrives/FY22_市場開発部IS_学生作業用/test_data.csv')
df = df[['Label', 'Comment']]
print(df)
result_list = []

for data in zip(df['Label'], df['Comment']):
  result = []
  label_num = 0
  if data[0] == '◎':
    label_num = 1.0
  elif data[0] == '○':
    label_num = 0.5
  elif data[0] == '□':
    label_num = 0
  elif data[0] == '▲':
    label_num = -0.5
  elif data[0] == '×':
    label_num = -1.0
  result.append(label_num)
  comment_vec = list(comment_to_vec(str(data[1]))) # n次元の固有ベクトルをリストで出力
  result += comment_vec
  result_list.append(result)
  # print(result)

result_df = pd.DataFrame(result_list) # result_listをdataframeに変換

result_df.to_csv('/content/drive/MyDrive/test_data_set1.csv')

In [ ]:
!pip install keras 
!pip install tensorflow

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

# csv読み込む場合のやつ
# 教師データ取得
train_data = pd.read_csv('/content/drive/Shareddrives/FY22_市場開発部IS_学生作業用/train_data_set1.csv', index_col=0)
# print(train_data)
x_train = train_data.iloc[:, 1:101]
y_train = train_data['0']
# print(x_train)
# print(y_train)

# テストデータ取得
test_data = pd.read_csv('/content/drive/Shareddrives/FY22_市場開発部IS_学生作業用//test_data_set1.csv', index_col=0)
# print(test_data)
x_test = test_data.iloc[:, 1:101]
y_test = test_data['0']

# dataframeから直接読むのやつ
# train_data = pd.concat([train_result_df, train_result_df_r]) # 水増しデータをくっつける場合
# train_data = train_result_d # 通常時
# x_train = train_data.iloc[:, 1:101]
# y_train = train_data[0]
# print(train_data)
# print(x_train)
# print(y_train)

# test_data = test_result_df
# x_test = test_data.iloc[:, 1:101]
# y_test = test_data[0

In [ ]:
#ニューラルネットワーク
#中間層のニューロン数

model = Sequential()

#活性化関数
model.add(Dense(64, activation='swish', input_dim=100))
model.add(Dense(32, activation='swish'))
model.add(Dense(64, activation='swish',))
model.add(Dense(1, activation='linear'))

#最適化アルゴリズム
model.compile(optimizer='rmsprop',
          loss='mean_squared_error',
          metrics=['accuracy'])

#モデルの学習
train_history = model.fit(x_train, y_train,
        batch_size=2048,
        epochs=20,
        verbose=1)

#損失値関数のプロット
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(train_history.history['loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('loss')
ax.set_xlim(0, 20)
ax.set_ylim(-0.01, 0.5)
# plt.savefig('nn-regression-train-1.png', dpi=300, facecolor='white')
plt.show()

#予測
y_pred = model.predict(x_test)
                       
#決定係数計算

from sklearn.metrics import mean_squared_error, r2_score
#R2
r2=r2_score(y_test, y_pred)

#RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R2:",r2,"RMSE:",rmse)

In [ ]:
# 一番良いモデルのみ保存
model.save('/content/drive/MyDrive/models/best_model')

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# ニューラルネットワーク(ループ版)

#中間層のニューロン数
# nn1 = 64
# nn2 = 32
model = Sequential()
#活性化関数
model.add(Dense(64, activation='swish', input_dim=100))
model.add(Dense(32, activation='swish'))
model.add(Dense(64, activation='swish',))
model.add(Dense(1, activation='linear'))

#最適化アルゴリズム
model.compile(optimizer='rmsprop',
          loss='mean_squared_error',
          metrics=['accuracy'])

def fit():
  train_history = model.fit(x_train, y_train,
          batch_size=2048,
          epochs=20,
          verbose=0)
  # 予測
  y_pred = model.predict(x_test)
  
  #R2
  r2=r2_score(y_test, y_pred)
  
  #RMSE
  # rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  return r2

output_list = []
max = 0.3227

for s in range(10):
  r2 = fit() 
  if r2 > max:
    max = r2
    model.save(f'/content/drive/MyDrive/models/model_{max}') # 良いモデルの保存(ファイル名はmode_決定係数)
  print(s, r2)
  output_list.append(r2)
  
mean = np.mean(output_list)

print(f'\naverage={mean}')

print(f'\nmax={max}')

output_list.append(mean)
output = pd.DataFrame(output_list)
output.to_csv("/content/drive/MyDrive/result_sample2.csv") # R^2の保存(最終行はR^2の平均値)

In [ ]:
# 既存モデル読み込み&実行
# exist_model = load_model('/content/drive/Shareddrives/FY22_市場開発部IS_学生作業用/good_model.hdf5')
exist_model = load_model('/content/drive/MyDrive/models/model_0.3226864519183694(swish, 64 32 64 16)')
y_pred = exist_model.predict(x_test)
r2=r2_score(y_test, y_pred)
print(r2)